In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
warehouse_path = '/content/drive/Shareddrives/SupplyChain/supply_chain_project/warehouses.xlsx'

data_warehouse = pd.read_excel(warehouse_path)
data_warehouse.head()

,Warehouse,Latitude,Longitude,City,State,Region
0,W1,27.59137,76.43122,Roondh Rampur,Rajasthan,North Central
1,W2,26.85383,84.19225,Amwas Khas,Uttar Pradesh,North Central
2,W3,17.20569,81.53633,Rajahmundry,Andhra Pradesh,South East
3,W4,28.56098,80.49032,Sumer Nagar,Uttar Pradesh,North Central
4,W5,29.25725,77.91551,Bhoomma,Uttar Pradesh,North Central


In [ ]:
data_warehouse[data_warehouse['Region'] == 'North']

,Warehouse,Latitude,Longitude,City,State,Region
5,W6,31.91242,76.05651,Kasba,Himachal Pradesh,North
34,W35,34.06834,74.79684,Batmaloo,Srinagar,North
51,W52,31.16756,77.65145,Shimla,Himachal Pradesh,North


In [ ]:
import folium

# Latitude and longitude data
data = [
    {'Warehouse': 'W6', 'Latitude': 31.91242, 'Longitude': 76.05651},
    {'Warehouse': 'W35', 'Latitude': 34.06834, 'Longitude': 74.79684},
    {'Warehouse': 'W52', 'Latitude': 31.16756, 'Longitude': 77.65145}
]

# Create a folium map centered around the centroid
centroid_latitude = 32.374351518704046
centroid_longitude = 76.1833314650945
map = folium.Map(location=[centroid_latitude, centroid_longitude], zoom_start=7)

# Add markers for each warehouse
for d in data:
    warehouse = d['Warehouse']
    latitude = d['Latitude']
    longitude = d['Longitude']
    folium.Marker([latitude, longitude], popup=warehouse).add_to(map)

# Add a marker for the centroid with a different color
folium.Marker(
    [centroid_latitude, centroid_longitude],
    popup='Centroid',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(map)

# Display the map
map


## Calculate the centroids of distributors based on warehouses locations

In [ ]:
import numpy as np

def calculate_geographic_centroid(coords):
    # Convert latitude and longitude to 
    # spherical coordinates in radians.
    degrees_to_radians = np.pi/180.0

    # Separate latitudes and longitudes into their own lists
    latitudes, longitudes = zip(*coords)

    # phi = 90 - latitude
    phi = (90.0 - np.array(latitudes))*degrees_to_radians

    # theta = longitude
    theta = np.array(longitudes)*degrees_to_radians

    # Compute spherical coordinate of centroid.
    cphi = np.cos(phi)
    sphi = np.sin(phi)
    ctheta = np.cos(theta)
    stheta = np.sin(theta)

    x = ctheta * sphi
    y = stheta * sphi
    z = cphi

    # Compute averages
    x_bar = np.mean(x)
    y_bar = np.mean(y)
    z_bar = np.mean(z)

    # Convert averages to latitude and longitude.
    centroid_latitude = 90.0 - np.arccos(z_bar)*180.0/np.pi
    centroid_longitude = np.arctan2(y_bar, x_bar)*180.0/np.pi

    return (centroid_latitude, centroid_longitude)


In [ ]:
def calculate_centroids(df):
    # Create an empty dictionary to store the centroids
    centroids = {}

    # Group the DataFrame by 'Region' and loop through each group
    for region, group in df.groupby('Region'):
        #print(region)
        #print(group)
        # Convert the Latitude and Longitude columns to a list of tuples
        coords = [(lat, lon) for lat, lon in zip(group['Latitude'], group['Longitude'])]

        # Calculate the centroid of the coordinates
        centroid = calculate_geographic_centroid(coords)

        # Store the centroid in the dictionary
        centroids[region] = centroid

    return centroids

centroids = calculate_centroids(data_warehouse)
for region, centroid in centroids.items():
    print(f"Centroid of {region}: {centroid}")



Centroid of Central: (22.838562467570583, 80.13168272188055)
Centroid of East: (24.28367059252669, 86.71595479685799)
Centroid of North: (32.374351518704046, 76.1833314650945)
Centroid of North Central: (26.938927380382275, 78.29771921958431)
Centroid of North East: (25.148890408139522, 91.91937757332543)
Centroid of North West: (22.855943785525824, 72.7710594887368)
Centroid of South East: (13.841481423282374, 80.78571515514709)
Centroid of South West: (14.054830636217162, 76.76216770440867)
Centroid of West: (19.948131831565618, 76.77202447425074)


In [ ]:
# Split dictionary values (tuples) into two separate dictionaries
centroid_latitude = {region: centroid[0] for region, centroid in centroids.items()}
centroid_longitude = {region: centroid[1] for region, centroid in centroids.items()}

# Add new columns to the DataFrame
data_warehouse['Centroid_Latitude'] = data_warehouse['Region'].map(centroid_latitude)
data_warehouse['Centroid_Longitude'] = data_warehouse['Region'].map(centroid_longitude)


In [ ]:
data_warehouse.head()

,Warehouse,Latitude,Longitude,City,State,Region,Centroid_Latitude,Centroid_Longitude
0,W1,27.59137,76.43122,Roondh Rampur,Rajasthan,North Central,26.938927,78.297719
1,W2,26.85383,84.19225,Amwas Khas,Uttar Pradesh,North Central,26.938927,78.297719
2,W3,17.20569,81.53633,Rajahmundry,Andhra Pradesh,South East,13.841481,80.785715
3,W4,28.56098,80.49032,Sumer Nagar,Uttar Pradesh,North Central,26.938927,78.297719
4,W5,29.25725,77.91551,Bhoomma,Uttar Pradesh,North Central,26.938927,78.297719


In [ ]:
import folium

# Create a map centered at the centroid latitude and longitude
map_center = [26.938927, 78.297719]
m = folium.Map(location=map_center, zoom_start=6)

# Add markers for the warehouses
warehouses = [
    {"name": "W1", "latitude": 27.59137, "longitude": 76.43122},
    {"name": "W2", "latitude": 26.85383, "longitude": 84.19225}
]

for warehouse in warehouses:
    name = warehouse["name"]
    latitude = warehouse["latitude"]
    longitude = warehouse["longitude"]
    tooltip = f"Warehouse {name}"
    folium.Marker([latitude, longitude], tooltip=tooltip).add_to(m)

# Display the map
m


In [ ]:
# import geopy.distance

# coords_1 = (52.2296756, 21.0122287)
# coords_2 = (52.406374, 16.9251681)
# coords_3 = (52.520008, 13.404954)

# print(geopy.distance.geodesic(coords_2, coords_3).km)

In [ ]:
plant_path = '/content/drive/Shareddrives/SupplyChain/supply_chain_project/plant_location.xlsx'

data_plant = pd.read_excel(plant_path)
data_plant

,plant_location,category,latitude,longitude
0,Jalgaon,Edible oils,21.0077,75.5626
1,Baddi,Edible oils,30.9569,76.7914
2,Perundurai,CNO,11.2753,77.5907
3,Kanjikode,CNO,10.7905,76.6513
4,Pondicherry,CNO,11.9416,79.8083
5,Sanand,VAHO,22.9920,72.3811
6,Guwahati,VAHO,26.1445,91.7362
7,Dehradun,VAHO,30.3165,78.0322


## Calculate the cost from plants to warehouses

In [ ]:
import pandas as pd
from geopy.distance import geodesic

distances = []
primary_costs = []

cost_factors = {
    'Guwahati': (0.002, 2.73),
    'Dehradun': (0.0052, 3.84),
    'Baddi': (0.0052, 3.84),
    'Jalgaon': (0.0048, 2.52),
    'Kanjikode': (0.0042, 1.64),
    'Perundurai': (0.0043, 1.76),
    'Pondicherry': (0.0044, 2.1),
    'Sanand': (0.005, 2.85)
}

# Loop over data_plant rows
for plant_location, plant_lat, plant_lon in zip(data_plant['plant_location'], data_plant['latitude'], data_plant['longitude']):
    plant_coords = (plant_lat, plant_lon)
    p_to_w_costs = []

    # For each plant, loop over data_warehouse rows
    for warehouse, warehouse_lat, warehouse_lon in zip(data_warehouse['Warehouse'], data_warehouse['Latitude'], data_warehouse['Longitude']):
        warehouse_coords = (warehouse_lat, warehouse_lon)

        # Calculate distance and append to list along with the warehouse and plant_location
        distance = round(geodesic(plant_coords, warehouse_coords).km, 2)
        distances.append((plant_location, warehouse, distance))

        # Calculate cost based on location
        if plant_location in cost_factors:
            cost_factor, constant = cost_factors[plant_location]
            p_to_w_cost = distance * cost_factor + constant
            p_to_w_costs.append(p_to_w_cost)

    primary_costs.append(p_to_w_costs)

# Create a dataframe from the costs list
df_primary_costs = pd.DataFrame(primary_costs, columns=data_warehouse['Warehouse'], index=data_plant['plant_location'])
df_primary_costs

Warehouse,W1,W2,W3,W4,W5,W6,W7,W8,W9,W10,...,W44,W45,W46,W47,W48,W49,W50,W51,W52,W53
plant_location,,,,,,,,,,,,,,,,,,,,,
Jalgaon,6.045744,7.755744,6.150288,7.192896,7.051536,8.324208,7.001904,2.930544,9.173760,12.788208,...,3.730848,8.455584,4.826784,11.572080,6.102480,10.760832,4.069152,5.025072,8.014896,3.721104
Baddi,5.788284,8.274300,12.145232,6.156652,4.970168,4.499932,14.203236,9.812200,11.535688,13.515172,...,10.897284,15.671612,9.637220,12.477876,13.419492,11.559296,9.824992,7.402936,4.283768,8.391560
Perundurai,9.544290,9.749959,5.122987,10.090691,10.321945,11.609795,2.671385,6.239482,9.347436,12.858515,...,5.575089,2.828851,6.706290,11.846553,3.297207,11.353515,6.272721,8.708714,11.231352,7.387195
Kanjikode,9.452042,9.814250,5.353976,10.072046,10.244582,11.466530,2.947250,6.203006,9.505592,12.942536,...,5.467124,3.070268,6.803522,11.947934,3.283376,11.454980,6.333920,8.555846,11.126204,7.343096
Pondicherry,9.879244,9.638740,4.790468,10.201940,10.579592,11.976768,2.957032,6.633760,8.943100,12.495748,...,6.194992,2.756040,6.692720,11.486212,4.058616,11.025268,6.401308,9.235392,11.518024,7.719944
Sanand,6.112700,9.183750,8.607100,7.951900,7.287400,8.114550,9.217600,5.253300,11.269900,14.724200,...,5.481950,10.752050,6.967850,13.486500,8.115050,12.594250,6.327000,4.349200,8.077450,5.001200
Guwahati,5.786600,4.242080,5.621580,5.017920,5.540280,6.037200,6.758760,6.120940,3.637900,3.521200,...,6.561360,6.996820,5.312280,3.013020,6.844720,2.893780,5.662040,6.344580,5.697380,5.873260
Dehradun,5.607792,7.553996,11.614624,5.440508,4.453392,5.184200,13.800600,9.512472,10.816424,12.855084,...,10.663076,15.232576,9.115036,11.793972,13.085236,10.868736,9.386736,7.428988,4.365980,8.088088


In [ ]:
#add a zero column named W0, so we can use df_primary_costs(i,j) represents cost from i to j warehouse,and we number the plant 0-7
import pandas as pd

# Insert a column filled with zeros before W1
df_primary_costs.insert(df_primary_costs.columns.get_loc("W1"), "W0", 0)

df_primary_costs

Warehouse,W0,W1,W2,W3,W4,W5,W6,W7,W8,W9,...,W44,W45,W46,W47,W48,W49,W50,W51,W52,W53
plant_location,,,,,,,,,,,,,,,,,,,,,
Jalgaon,0,6.045744,7.755744,6.150288,7.192896,7.051536,8.324208,7.001904,2.930544,9.173760,...,3.730848,8.455584,4.826784,11.572080,6.102480,10.760832,4.069152,5.025072,8.014896,3.721104
Baddi,0,5.788284,8.274300,12.145232,6.156652,4.970168,4.499932,14.203236,9.812200,11.535688,...,10.897284,15.671612,9.637220,12.477876,13.419492,11.559296,9.824992,7.402936,4.283768,8.391560
Perundurai,0,9.544290,9.749959,5.122987,10.090691,10.321945,11.609795,2.671385,6.239482,9.347436,...,5.575089,2.828851,6.706290,11.846553,3.297207,11.353515,6.272721,8.708714,11.231352,7.387195
Kanjikode,0,9.452042,9.814250,5.353976,10.072046,10.244582,11.466530,2.947250,6.203006,9.505592,...,5.467124,3.070268,6.803522,11.947934,3.283376,11.454980,6.333920,8.555846,11.126204,7.343096
Pondicherry,0,9.879244,9.638740,4.790468,10.201940,10.579592,11.976768,2.957032,6.633760,8.943100,...,6.194992,2.756040,6.692720,11.486212,4.058616,11.025268,6.401308,9.235392,11.518024,7.719944
Sanand,0,6.112700,9.183750,8.607100,7.951900,7.287400,8.114550,9.217600,5.253300,11.269900,...,5.481950,10.752050,6.967850,13.486500,8.115050,12.594250,6.327000,4.349200,8.077450,5.001200
Guwahati,0,5.786600,4.242080,5.621580,5.017920,5.540280,6.037200,6.758760,6.120940,3.637900,...,6.561360,6.996820,5.312280,3.013020,6.844720,2.893780,5.662040,6.344580,5.697380,5.873260
Dehradun,0,5.607792,7.553996,11.614624,5.440508,4.453392,5.184200,13.800600,9.512472,10.816424,...,10.663076,15.232576,9.115036,11.793972,13.085236,10.868736,9.386736,7.428988,4.365980,8.088088


In [ ]:
df_primary_costs.iloc[0,1]
for i in range(2,5):
  print(i)

2
3
4


## Calculate the cost from warehouses to distributors

In [ ]:
data_distributor = data_warehouse[['Region','Centroid_Latitude','Centroid_Longitude']].drop_duplicates().reset_index(drop=True)
data_distributor

,Region,Centroid_Latitude,Centroid_Longitude
0,North Central,26.938927,78.297719
1,South East,13.841481,80.785715
2,North,32.374352,76.183331
3,South West,14.054831,76.762168
4,West,19.948132,76.772024
5,East,24.283671,86.715955
6,North East,25.148890,91.919378
7,North West,22.855944,72.771059
8,Central,22.838562,80.131683


In [ ]:
import pandas as pd
from geopy.distance import geodesic

secondary_costs = []
secondary_distance = []
cost_factors = {
    'North East': 0.0145,
    'North': 0.011875,
    'West': 0.0115,
    'South West': 0.010625,
    'South East': 0.00915,
    'Central': 0.008875,
    'East': 0.008,
    'North Central': 0.007875,
    'North West': 0.006875
}

# Loop over data_warehouse rows
for warehouse, warehouse_lat, warehouse_lon in zip(data_warehouse['Warehouse'], data_warehouse['Latitude'], data_warehouse['Longitude']):
    warehouse_coords = (warehouse_lat, warehouse_lon)
    w_to_d_costs = []
    w_to_d_distances = []
    # For each warehouse, loop over data_distributor rows
    for distributor_location, distributor_lat, distributor_lon in zip(data_distributor['Region'], data_distributor['Centroid_Latitude'], data_distributor['Centroid_Longitude']):
        distributor_coords = (distributor_lat, distributor_lon)

        # Calculate distance and append to list along with the warehouse and distributor_location
        distance = round(geodesic(distributor_coords, warehouse_coords).km, 2)

        # Calculate cost based on location
        if distance > 500:
            w_to_d_cost = 9999
        elif distributor_location in cost_factors:
            slope = cost_factors[distributor_location]
            w_to_d_cost = distance * slope
        else:
            w_to_d_cost = None
        w_to_d_distances.append(distance)
        w_to_d_costs.append(w_to_d_cost)

    secondary_costs.append(w_to_d_costs)
    secondary_distance.append(w_to_d_distances)
# Create a dataframe from the costs list
df_secondary_costs = pd.DataFrame(secondary_costs, columns=data_distributor['Region'], index=data_warehouse['Warehouse'])
df_secondary_disntance = pd.DataFrame(secondary_distance, columns=data_distributor['Region'], index=data_warehouse['Warehouse'])
# Create a DataFrame with the zero row
zero_row = pd.DataFrame([[0] * len(df_secondary_costs.columns)], columns=df_secondary_costs.columns)

# Concatenate the zero row with the original DataFrame
df_secondary_costs = pd.concat([zero_row, df_secondary_costs])
df_secondary_disntance = pd.concat([zero_row, df_secondary_disntance])
df_secondary_disntance
#df_secondary_costs

Region,North Central,South East,North,South West,West,East,North East,North West,Central
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
W1,198.46,1588.04,530.74,1499.14,847.25,1092.98,1568.14,641.15,645.07
W2,585.62,1483.63,987.45,1614.12,1076.57,381.25,796.18,1235.68,605.10
W3,1128.25,380.87,1764.52,619.31,587.34,951.07,1390.32,1109.58,640.66
W4,281.12,1630.09,591.30,1651.77,1025.78,780.96,1196.46,999.33,634.97
W5,259.65,1732.45,383.26,1687.56,1037.62,1033.73,1459.03,876.06,744.85
W6,592.58,2058.60,52.60,1979.04,1327.44,1344.76,1721.51,1054.67,1082.99
W7,1540.84,274.57,2155.13,210.32,785.83,1523.76,1953.81,1240.34,1102.06
W8,731.95,889.32,1304.47,726.46,89.86,1150.40,1682.53,438.96,470.85
W9,1146.15,1324.01,1617.46,1613.24,1293.57,255.09,392.88,1645.00,890.97


In [ ]:
df_secondary_costs

Region,North Central,South East,North,South West,West,East,North East,North West,Central
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
W1,1.562873,9999.000000,9999.000000,9999.000000,9999.000000,9999.00000,9999.000000,9999.000000,9999.000000
W2,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,3.05000,9999.000000,9999.000000,9999.000000
W3,9999.000000,3.484961,9999.000000,9999.000000,9999.000000,9999.00000,9999.000000,9999.000000,9999.000000
W4,2.213820,9999.000000,9999.000000,9999.000000,9999.000000,9999.00000,9999.000000,9999.000000,9999.000000
W5,2.044744,9999.000000,4.551213,9999.000000,9999.000000,9999.00000,9999.000000,9999.000000,9999.000000
W6,9999.000000,9999.000000,0.624625,9999.000000,9999.000000,9999.00000,9999.000000,9999.000000,9999.000000
W7,9999.000000,2.512316,9999.000000,2.234650,9999.000000,9999.00000,9999.000000,9999.000000,9999.000000
W8,9999.000000,9999.000000,9999.000000,9999.000000,1.033390,9999.00000,9999.000000,3.017850,4.178794
W9,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,2.04072,5.696760,9999.000000,9999.000000


In [ ]:
print(w_to_d_distances)

[463.44, 1112.69, 1032.95, 996.93, 345.11, 1042.76, 1575.91, 391.06, 365.28]


In [ ]:
df_secondary_costs.iloc[1,0]

1.5628725

In [ ]:
demand = {
    "North Central": 134560,
    "South East": 134560,
    "North": 8410,
    "South West": 75690,
    "West": 168200,
    "East": 117740,
    "North East": 25230,
    "North West": 92510,
    "Central": 84100
}

In [ ]:
D_distributor = np.array(list(demand.values()))

# Print the demand array
print(D_distributor)
total_demand = sum(D_distributor)
total_demand

[134560 134560   8410  75690 168200 117740  25230  92510  84100]


841000

In [ ]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 83.9 MB/s eta 0:00:00


**select 24 warehouses**

In [ ]:
import re
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, value, LpAffineExpression, LpStatus, PULP_CBC_CMD

non_zero_s_list = []
# Create the problem
prob = LpProblem("Cost Minimization Problem", LpMinimize)

# Define the decision variables
plants = range(8)
warehouses = range(1, 54)
distributors = range(9)

# Decision variables
W = LpVariable.dicts("W", ((p, w, d) for p in plants for w in warehouses for d in distributors), lowBound=0, cat="Integer")
s = LpVariable.dicts("s", ((w, d) for w in warehouses for d in distributors), lowBound=0, cat="Integer")
#Y = LpVariable.dicts("Y", warehouses, cat="Binary")




# Objective function
prob += lpSum(df_primary_costs.iloc[p, w] * W[p, w, d] for p in range(8) for w in range(1,54) for d in range(9)) \
        + lpSum(df_secondary_costs.iloc[w, d] * s[w, d] for w in range(1,54) for d in range(9))



# Constraints
total_demand = lpSum(D_distributor[d] for d in range(9))


# No delivery for distances larger than 500km
for w in range(1,54):
    for d in range(9):
        if df_secondary_costs.iloc[w, d] > 500:
            prob += s[w, d] == 0

#prob += lpSum([i[1] for i in W]) == 24



#category
for d in distributors:
  # Edible = 30%
  prob += lpSum(W[p, w, d] for p in range(2) for w in range(1,54)) == D_distributor[d] * 0.3

  # VAHO = 24%
  prob += lpSum(W[p, w, d] for p in range(2, 5) for w in range(1,54)) == D_distributor[d] * 0.24

  # CNO = 46%
  prob += lpSum(W[p, w, d] for p in range(5, 8) for w in range(1,54)) == D_distributor[d] * 0.46

# Inflow equals outflow
prob += lpSum(W[p, w, d] for p in range(8) for w in range(1, 54) for d in range(9)) \
        == lpSum(s[w, d] for w in range(1, 54) for d in range(9))

# Regional demand
for d in range(9):
    prob += lpSum(s[w, d] for w in range(1,54)) == D_distributor[d]
for w in range(1,54):
  prob += lpSum(W[p, w, d] for p in range(8) for d in range(9)) \
        == lpSum(s[w, d] for d in range(9))

#capacity
for p in plants:
  for w in warehouses:
        prob += lpSum(W[p, w, d] for d in distributors) <= 28034

# Solve the problem
prob.solve(PULP_CBC_CMD())

# Print the optimal solution
print("Optimal Solution:")
for v in prob.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)

# Print the optimal cost
print("Total Cost =", value(prob.objective))
non_zero_s = sum(1 for v in prob.variables() if v.name.startswith('s_') and v.varValue > 0)


indexes_w = []
indexes_d = []

for v in prob.variables():
    if v.name.startswith('s_') and v.varValue > 0:
        indexes = v.name.split('_')[1].split(',')
        w_index = int(indexes[0].strip('('))
        d_index = re.search(r"s_\(\d+,_([\d]+)\)", v.name).group(1) 
        indexes_w.append(w_index)
        indexes_d.append(d_index)

# Print the arrays
print("Warehouse indexes:", indexes_w)
indexes_d = numbers = [int(index) for index in indexes_d]

print("Distributor indexes:", indexes_d)





/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Optimal Solution:
W_(0,_11,_5) = 28034.0
W_(0,_25,_4) = 12894.0
W_(0,_25,_7) = 15140.0
W_(0,_31,_0) = 11210.0
W_(0,_37,_1) = 11493.0
W_(0,_37,_4) = 9251.0
W_(0,_37,_5) = 7288.0
W_(0,_42,_4) = 28034.0
W_(0,_44,_0) = 5327.0
W_(0,_44,_3) = 22707.0
W_(0,_48,_0) = 8410.0
W_(0,_50,_1) = 20465.0
W_(0,_50,_6) = 7569.0
W_(0,_8,_0) = 15421.0
W_(0,_8,_7) = 12613.0
W_(1,_19,_2) = 2523.0
W_(1,_19,_4) = 281.0
W_(1,_19,_8) = 25230.0
W_(1,_6,_1) = 8410.0
W_(2,_18,_1) = 4260.4
W_(2,_18,_5) = 4931.2
W_(2,_18,_8) = 2020.4
W_(2,_20,_0) = 28034.0
W_(2,_29,_0) = 4260.4
W_(2,_29,_8) = 18163.6
W_(2,_48,_2) = 2018.4
W_(2,_48,_4) = 2689.2
W_(2,_48,_5) = 23326.4
W_(3,_20,_1) = 28034.0
W_(3,_48,_4) = 28034.0
W_(4,_20,_3) = 18165.6
W_(4,_20,_6) = 6055.2
W_(4,_20,_7) = 3813.2
W_(4,_29,_4) = 9644.8
W_(4,_29,_7) = 18389.2
W_(5,_11,_0) = 8408.0
W_(5,_16,_8) = 28034.0
W_(5,_26,_0) = 14241.6
W_(5,_26,_3) = 11552.4
W_(5,_26,_8) = 2240.0
W_(5,_8,_5) = 28034.0
W_(6,_19,_0) = 28034.0
W_(6,_23,_5) = 19622.0
W_(6,_23,_8) = 84

In [ ]:
each_distances_list = []
for i in range(len(indexes_w)):
    each_distances = df_secondary_disntance.iloc[indexes_w[i],indexes_d[i]]
    #print(each_distances)
    each_distances_list.append(each_distances)

less_than_250 = 0
more_than_250 = 0
for i in each_distances_list:
  if i <= 250:
    less_than_250 += 1
  else:
    more_than_250 += 1

print(less_than_250)
print(more_than_250)
service_level = less_than_250/(less_than_250+more_than_250)
print(service_level)

20
4
0.8333333333333334


**Select 22 Warehouses**

In [ ]:
import re
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, value, LpAffineExpression, LpStatus, PULP_CBC_CMD

non_zero_s_list = []
# Create the problem
prob = LpProblem("Cost Minimization Problem", LpMinimize)

# Define the decision variables
plants = range(8)
warehouses = range(1, 54)
distributors = range(9)

# Decision variables
W = LpVariable.dicts("W", ((p, w, d) for p in plants for w in warehouses for d in distributors), lowBound=0, cat="Integer")
s = LpVariable.dicts("s", ((w, d) for w in warehouses for d in distributors), lowBound=0, cat="Integer")
#Y = LpVariable.dicts("Y", warehouses, cat="Binary")




# Objective function
prob += lpSum(df_primary_costs.iloc[p, w] * W[p, w, d] for p in range(8) for w in range(1,54) for d in range(9)) \
        + lpSum(df_secondary_costs.iloc[w, d] * s[w, d] for w in range(1,54) for d in range(9))



# Constraints
total_demand = lpSum(D_distributor[d] for d in range(9))


# No delivery for distances larger than 500km
for w in range(1,54):
    for d in range(9):
        if df_secondary_costs.iloc[w, d] > 500:
            prob += s[w, d] == 0

#prob += lpSum([i[1] for i in W]) == 24



#category
for d in distributors:
  # Edible = 30%
  prob += lpSum(W[p, w, d] for p in range(2) for w in range(1,54)) == D_distributor[d] * 0.3

  # VAHO = 24%
  prob += lpSum(W[p, w, d] for p in range(2, 5) for w in range(1,54)) == D_distributor[d] * 0.24

  # CNO = 46%
  prob += lpSum(W[p, w, d] for p in range(5, 8) for w in range(1,54)) == D_distributor[d] * 0.46

# Inflow equals outflow
prob += lpSum(W[p, w, d] for p in range(8) for w in range(1, 54) for d in range(9)) \
        == lpSum(s[w, d] for w in range(1, 54) for d in range(9))

# Regional demand
for d in range(9):
    prob += lpSum(s[w, d] for w in range(1,54)) == D_distributor[d]
for w in range(1,54):
  prob += lpSum(W[p, w, d] for p in range(8) for d in range(9)) \
        == lpSum(s[w, d] for d in range(9))

#capacity
for p in plants:
  for w in warehouses:
        prob += lpSum(W[p, w, d] for d in distributors) <= 33000

# Solve the problem
prob.solve(PULP_CBC_CMD())

# Print the optimal solution
print("Optimal Solution:")
for v in prob.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)

# Print the optimal cost
print("Total Cost =", value(prob.objective))
non_zero_s = sum(1 for v in prob.variables() if v.name.startswith('s_') and v.varValue > 0)


indexes_w = []
indexes_d = []

for v in prob.variables():
    if v.name.startswith('s_') and v.varValue > 0:
        indexes = v.name.split('_')[1].split(',')
        w_index = int(indexes[0].strip('('))
        d_index = re.search(r"s_\(\d+,_([\d]+)\)", v.name).group(1) 
        indexes_w.append(w_index)
        indexes_d.append(d_index)

# Print the arrays
print("Warehouse indexes:", indexes_w)
indexes_d = numbers = [int(index) for index in indexes_d]

print("Distributor indexes:", indexes_d)





/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Optimal Solution:
W_(0,_11,_5) = 19380.0
W_(0,_25,_4) = 16582.0
W_(0,_25,_7) = 16418.0
W_(0,_37,_1) = 2158.0
W_(0,_37,_5) = 15942.0
W_(0,_42,_1) = 4369.0
W_(0,_42,_4) = 28631.0
W_(0,_44,_0) = 10293.0
W_(0,_44,_3) = 22707.0
W_(0,_48,_0) = 8410.0
W_(0,_50,_1) = 25431.0
W_(0,_50,_6) = 7569.0
W_(0,_8,_0) = 21665.0
W_(0,_8,_7) = 11335.0
W_(1,_19,_2) = 2523.0
W_(1,_19,_4) = 5247.0
W_(1,_19,_8) = 25230.0
W_(1,_6,_1) = 8410.0
W_(2,_18,_8) = 1280.0
W_(2,_20,_0) = 31588.8
W_(2,_20,_8) = 1411.2
W_(2,_29,_8) = 2560.0
W_(2,_48,_2) = 2018.4
W_(2,_48,_4) = 2724.0
W_(2,_48,_5) = 28257.6
W_(3,_20,_0) = 705.6
W_(3,_20,_1) = 32294.4
W_(3,_48,_4) = 18067.2
W_(3,_48,_8) = 14932.8
W_(4,_20,_3) = 18165.6
W_(4,_20,_6) = 6055.2
W_(4,_20,_7) = 8779.2
W_(4,_29,_4) = 19576.8
W_(4,_29,_7) = 13423.2
W_(5,_11,_0) = 7130.0
W_(5,_16,_8) = 33000.0
W_(5,_26,_0) = 19207.6
W_(5,_26,_3) = 13792.4
W_(5,_8,_3) = 1247.2
W_(5,_8,_5) = 20147.0
W_(5,_8,_6) = 11605.8
W_(6,_19,_0) = 33000.0
W_(6,_23,_2) = 668.6
W_(6,_23,_5) = 2664

In [ ]:
each_distances_list = []
for i in range(len(indexes_w)):
    each_distances = df_secondary_disntance.iloc[indexes_w[i],indexes_d[i]]
    #print(each_distances)
    each_distances_list.append(each_distances)

less_than_250 = 0
more_than_250 = 0
for i in each_distances_list:
  if i <= 250:
    less_than_250 += 1
  else:
    more_than_250 += 1

print(less_than_250)
print(more_than_250)
service_level = less_than_250/(less_than_250+more_than_250)
print(service_level)

20
2
0.9090909090909091
